In [1]:
! pip install finance-datareader

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import os
import itertools
import random

import FinanceDataReader as fdr
from tqdm import tqdm
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
random.seed(1234)
np.random.seed(1234)
os.environ["PYTHONHASHSEED"] = str(1234)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 나눠서 GridSearch

In [5]:
path = '/content/drive/MyDrive/주식 종가 예측/open/'
list_name = 'stock_list.csv'
stock_list = pd.read_csv(os.path.join(path,list_name))
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))

##############################################################
###                       인덱스 바꿔주기                      ###
##############################################################
stock_list = stock_list[320:340].reset_index(drop = True)
##############################################################
###                       인덱스 바꿔주기                      ###
##############################################################

start_date = '20210104'
end_date = '20211126'

start_weekday = pd.to_datetime(start_date).weekday()
max_weeknum = pd.to_datetime(end_date).strftime('%V')
Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])

In [6]:
import warnings
import itertools
from statsmodels.tsa.statespace.sarimax import SARIMAX

# ignore warnings
warnings.filterwarnings("ignore")

for code in tqdm(stock_list['종목코드'].values):
  print(f'\n code: {code} \n')

  # Load data
  stock_data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
  stock_data.fillna(method = 'ffill', inplace = True)

  train = stock_data[:-20] # ~ 10/29

  # Set the parameter combinations
  p = d = q = range(0, 3)
  s = [0, 5]
  pdq = list(itertools.product(p, d, q))
  seasonal_pdqs = list(itertools.product(p, d, q, s))
  trend_params = ('c', 't')

  # Generate a dataframe to save values
  gridsearch_result = pd.DataFrame(columns = ['pdq', 'seasonal_pdqs', 'trend_param', 'aic'])

  # Grid Search
  for params in pdq:
    for seasonal_params in seasonal_pdqs:
      for trend_param in trend_params:
        try:
          model = SARIMAX(train['Close'].values.astype('float64'), exog = None, order = params, seasonal_order = seasonal_params, trend = trend_param, 
                          enforce_stationarity = True, enforce_invertibility = True)
          model_fit = model.fit()
          aic = model_fit.aic

          gridsearch_result = gridsearch_result.append({'pdq' : params, 'seasonal_pdqs' : seasonal_params, 'trend_param' : trend_param, 'aic' : aic}, ignore_index = True)
      
        except:
          continue
  
  # Save grid search results
  gridsearch_result.to_csv(f'/content/drive/MyDrive/주식 종가 예측/sarimax_gridsearch_final/{code}_sarimax_gridsearch.csv', index = False)

  0%|          | 0/20 [00:00<?, ?it/s]


 code: 216080 



  5%|▌         | 1/20 [39:12<12:25:04, 2352.87s/it]


 code: 013120 



 10%|█         | 2/20 [1:19:01<11:52:05, 2373.64s/it]

...

 90%|█████████ | 18/20 [11:44:25<1:18:35, 2357.57s/it]


 code: 347860 



 95%|█████████▌| 19/20 [12:22:11<38:50, 2330.10s/it]  


 code: 003800 



100%|██████████| 20/20 [12:58:17<00:00, 2334.85s/it]
